### P-2.36
Write a Python program to simulate an ecosystem containing two types of creatures, bears and ﬁsh.

The ecosystem consists of a river, which is modeled as a relatively large list.

Each element of the list should be a `Bear` object, a `Fish` object, or `None`.

In each time step, based on a random process, each animal either attempts to move into an adjacent list location or stay where it is.

- If two animals of the same type are about to collide in the same cell, then they stay where they are, but they create a new instance of that type of animal, which is placed in a random empty (i.e., previously `None`) location in the list.
- If a bear and a ﬁsh collide, however, then the ﬁsh dies (i.e., it disappears).

In [95]:
'''
river[target_position] = river[self_position] 
river[self_position] = None
'''        

l = list(['a', None])
print(l)
l[1] = l[0]
l[0] = None
print(l)

['a', None]
[None, 'a']


In [88]:
l[1] = l[0]

In [100]:
l[1]

'a'

In [6]:
# IMPORTS

import random
import functools


# LOGGER

import sys
from loguru import logger

logger.remove() # clear the decks
logger.add("debug.log", mode="w", backtrace=False) # specify mode="w" to overwrite each time, or a to append

logger.info("NEW RUN STARTED")

debug = False


# CONSTANTS

RIVER_LENGTH = 100
P_BEAR = 0.05
P_FISH = 0.1


def display_river(t):
    print(str(t).zfill(3), end=' ')
    for i, item in enumerate(river):
        if not item:
            print('-', end='')
        elif isinstance(item, Fish):
            print('f', end='') if item.get_is_born() and debug else print('F', end='')
        elif isinstance(item, Bear):
            print('\x1b[31mb\x1b[0m', end='') if item.get_is_born() and debug else print('\x1b[31mB\x1b[0m', end='')
    print('')

def find_free(l, pos):
    '''Returns index value of closest None value to a specified position in a list.
    Returns -1 if no None values are found'''
    
    offset, left_done, right_done, length = 0, False, False, len(l)
    
    while not (right_done and left_done):
        
        offset += 1
        
        pos_to_right = pos + offset
        pos_to_left = pos - offset
        
        if pos_to_right <= length-1:
            if not l[pos_to_right]:
                return pos_to_right
        else:
            right_done = True
        
        if pos_to_left >= 0:
            if not l[pos_to_left]:
                return pos_to_left
        else:
            left_done = True

    return -1

class Animal:
    def __init__(self, position, born = False):
        self._position = position
        self._processed = False
        self._born = born
        
    def get_processed(self):
        return self._processed
    
    def get_is_born(self):
        return self._born
    
    def set_processed(self):
        self._processed = True
    
    def reset_processed(self):
        self._processed = False

    def get_position(self):
        return self._position        

    def have_a_baby(self):
        """Creates a new instance of specified type in closest empty position to self.
        """
          
        empty_pos = find_free(river, self.get_position())
        
        if empty_pos == -1:
            global river_is_full
            river_is_full = True
            return
        
        if isinstance(self, Bear):
            if debug: logger.debug(f"Creating Bear with born=True at {empty_pos}")
            river[empty_pos] = Bear(empty_pos, born=True)
            river[empty_pos].set_processed()
            
        elif isinstance(self, Fish):
            if debug: logger.debug(f"Creating Fish with born=True at {empty_pos}")
            river[empty_pos] = Fish(empty_pos, born=True)
            river[empty_pos].set_processed()
        
    def __add__(self, other):
        """Operator override to allow straightforward addition of objects.
        Doesn't accept None as an argument"""
        
        self_position, other_position = self.get_position(), other.get_position()
        
        if type(self) == type(other):
            if debug: logger.debug(f"T{t}: Calling have_a_baby between {type(self), self_position} and {type(other), other_position}")
            self.have_a_baby()
            self.set_processed()
            other.set_processed()
            return "A new beast is born" # The creature is the same type

        elif isinstance(self, Bear): # bear eats fish
            
            if debug: logger.debug(f"T{t}: Bear at {self_position} eats fish at {other_position}")
            
            river[other_position] = river[self_position]
            
            river[self_position] = None
            river[other_position]._position = other_position
            river[other_position].set_processed()
                        
            return "There's been a murder" 

        elif isinstance(self, Fish): # fish moves in with bear, so disappears
            
            if debug: logger.debug(f"T{t}: Fish at {self_position} moves in with Bear, so is eaten")
            
            river[self_position] = None
            # self.set_processed() # Unecessary as we've removed ref to self from river
            
            return "Kamikaze"
            
    
    def move(self, direction):
        
        self_position = self.get_position()
        
        if debug: logger.debug(f"T{t}: Move from {self_position}, {direction}")
        
        if direction == 'upstream' and self_position == 0:
            return # Already at source. Can't move upstream
    
        elif direction == 'downstream' and self_position == len(river)-1:
            return # Already at mouth. Can't move downstream

        if direction == 'upstream':
            target_position = self_position - 1
        
        elif direction == 'downstream':
            target_position = self_position + 1

        if river[target_position]: # There's a creature in target location
            
            if debug: logger.debug(f"T{t}: Add {self_position} to {target_position}")
            
            river[self_position] + river[target_position]
        
        else: # There's a None in target location 
            
            river[target_position] = river[self_position]
            river[target_position]._position = target_position
            river[target_position].set_processed()
            river[self_position] = None
            
            
            if debug: logger.debug(f"T{t}: Move to {target_position}")
    
    def time_step(self):
        
        moves = bool(round(random.random())) # Does the creature move?
        
        if moves:
            up = bool(round(random.random())) # Does the creature move up or downstream?
            self.move('upstream') if up else self.move('downstream')
        
class Bear(Animal):
    def __init__(self, position, born = False):
        super().__init__(position, born = False)
        self._born = born

class Fish(Animal):
    def __init__(self, position, born = False):
        super().__init__(position, born = False)
        self._born = born

# Create empty river
river = [None] * RIVER_LENGTH
rivers = list()
river_is_full = False

log = []

# Populate river
for i in range(len(river)):
    r = random.choices(['bear', 'fish', None], [P_BEAR, P_FISH, 1-(P_BEAR + P_FISH)])[0]
    if r == 'bear':
        river[i] = Bear(i)
        #print("Creating Bear", i)
    elif r == 'fish':
        river[i] = Fish(i)
        #print("Creating Fish", i)
    else:
        pass

# Here we run our simulation

t = 0

def display_positions_labels():
    print("Tx ", "         ".join('0123456789'))

display_positions_labels()
display_river(t)

while not river_is_full:
    t += 1
    if t % 10 == 0 and debug: display_positions_labels()
    display_river(t) # t specifies the first character of the row
    for position in river: # iterate through all elements in river
        if position: # ... that aren't None
            #print(position)
            if not position.get_processed(): # ... that haven't been processed already
                    position.time_step()
            
    for position in river:
        if position:
            position.reset_processed()
            
    log.append(["----- COMPLETED TIME STEP -----"])      

display_river(t + 1)
print('River is full!')

Tx  0         1         2         3         4         5         6         7         8         9
000 --B--B-----------B-------F-----FF---------------------------F--------------------------F---F--B----F
001 --B--B-----------B-------F-----FF---------------------------F--------------------------F---F--B----F
002 ---BB------------B-------F----FFF---------------------------F--------------------------F---F-B----F-
003 --B-B------------B-------F---FF--F-------------------------F----------------------------F-F-B-----F-
004 --BB-------------B-------F--FFF---F-------------------------F---------------------------F--B------F-
005 -BBB------------B---------F-FFFF---F------------------------F---------------------------F--B-----F--
006 -BBB-------------B-------F-F-FF-F-F-------------------------F---------------------------F--B----F---
007 -BBBB-----------B--------F--FFFFFFFFF-----------------------F----------------------------FB-----F---
008 BBBBBBB---------B--------FFFFFFFFFFFFFF---------------------

__Follow up notes__

I spent a not-inconsiderable amount of time trying to work out why the following happens:
```
-F------
------F-
```

By capturing some rudimentary logs I identified the following

```
[['from: ', 23, 'downstream'],
 ['to: ', 24],
 ['from: ', 24, 'upstream'],
 ['to: ', 23],
 ['from: ', 33, 'downstream'],
 ['to: ', 34],
 ['from: ', 34, 'downstream'],
 ['to: ', 35],
 ['from: ', 35, 'downstream'],
 ['to: ', 36],
 ['from: ', 36, 'downstream'],
 ['to: ', 37],
 ['from: ', 37, 'downstream'],
 ['to: ', 38],
```

Here we can see that because our simulation iterates through each position in turn (`for i, position in enumerate(river)` - i.e. downstream), there are potential sequences of events that will cause multiple location changes while it would appear that only one should occur (33 -> 34, 34 -> 35 ... 37 -> 38!). Further, it's possible that a move downstream and then back upstream will occur within one timestep which looks like no movement in our representation (23 -> 24, 24 -> 23). The solution to this is to add in some kind of verification as to whether a particular object has already moved in the current time step, or to reorientate the simulation so that each object is iterated over, instead of each position in the river/list. Another soltuion would be to change the positions of each object after all positions had been processed.

One of the major hurdles in the project was the requirement to use a `list` as the representation of the river. This data structure is a bit limited as lists, by definition, support multiple identical items, so there's no method to identify each objects position without using some kind of tracking mechanism/property, and this got a little arduous and complicated. A `River` type/class, based on built-in `list` type/class would be the solution. This would also allow a neat representation which would improve on my `display_river()` method: simply calling the `River` object would cause a call to `__repr__()` which could return a sequence of `F`, `B`, or `-` as appropriate.

Also, working with `None` items within the list was a bit arduous too. A modified `None` type would allow operations such as `Bear + None` which would have made things more elegant: there are frequent checks (e.g. `if target:`) as to whether a position contains `None` to avoid problematic calls (e.g. `None.get_position()`).

I also stumbled when trying to keep a record of each timestep by appending `river` to a new list called `rivers` in the hope that I'd be able to view the state of river at differing timesteps by subscribing: `rivers[12]`. This doesn't work well at all, as `rivers.append(river)` results in aliases being established between e.g. `rivers[0]` and `river`, so when subsequently `river` changes, so does `rivers[0]`. Creating a `copy()` or a `deepcopy()` would help, but we'd need to make sure we can easily identify the individual objects, perhaps by exposing the position member data more readily.

Effectively, this is approaching something similar to a Turing machine!

I think I found another issue:

```
-F-B----
--FBB---
--FBBB--
```

```
['----- COMPLETED TIME STEP -----'],
 ['from: ', 12, 'downstream'], # fish
 ['to: ', 13],                 
 ['from: ', 15, 'downstream'], # bear
 ['to: ', 16],
 ['from: ', 16, 'upstream'],   # bear
 ['to: ', 15]
 ...
 ['----- COMPLETED TIME STEP -----'],
 ['from: ', 13, 'downstream'],
 ['from: ', 15, 'upstream'],
 ```
 
 In this instance, we're seeing a `Bear` spawn another without requiring another `Bear` to mate with! From exploring similar instances in subsequent runs, I established there were instances where objects of the same type were being added to themselves! This would happen if an object was moved, and then a subsequent comparison during the same time-step was processed on the moved object and on the position the object had moved from (in the case that the old position had not been set back to `None`).
 
 - Make sure to set the `river[self.get_position()]` to `None` following an operation which moves that object
 - Be aware of the difference of referring to `self` and to `river[self.get_position()]`
 
My method of creating a `processed` value for each item that has been manipulated already within a given timestep works quite well: rules like this make the system much more straightforward to debug.

The last observation I'd make is how important it is to reduce the complexity of the problem when troubleshooting: by changing the size of the river to just 10, and setting p(Bear) to 0 I was able to drill down (having implemented some debugging logs) and isolate strange occurences.